In [9]:
#Data source
dpath = "G:/Lire/"
import pandas as pd, numpy as np

#Preprocessing steps
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

#Metrics to use
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score, confusion_matrix

#Classifiers
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [10]:
#List of all classifiers that will be trained
classifiers = [LogisticRegression(), RandomForestClassifier(), GaussianNB()]

In [11]:
#List of all features that will be combined in late fusion
feat_to_combine = ["FCTH", "JCD", "Tamura"]

In [12]:
#Creation of the feature set list

#We'll load all the feature files and store them as tuples of train,test sets
#Also we'll create the folds division for the training set

n_folds = 3

feature_sets = {}
for feat in feat_to_combine:
    features_train = pd.read_csv(dpath + "features_%s_%s.csv" % (feat, "train"))
    features_test = pd.read_csv(dpath + "features_%s_%s.csv" % (feat, "test"))
    
    features_train["folds"] = np.random.choice(range(1, n_folds + 1), features_train.shape[0])
    
    feature_sets[feat] = (features_train, features_test)


In [13]:
#Function to obtain the classifier name from their string representation
def classifier_name(classifier):
    return str(classifier).split("(")[0]

In [14]:
#Function that creates predictions for each of the folds 
#we'll use one as testing set and the others as training set, storing the predictions of the current base model in new column in the training dataset
#If we add a scoring function we can obtain cross-validated metrics

def predict_by_folds(model, data, folds, target, metric_function=None): 
    
    folds_prediction = target.copy()
    metrics = [] if metric_function else None
    
    for f in folds.unique():
        train_i = np.array(folds != f)
        train1 = data[train_i, :]
        test1 = data[~train_i, :]
        y1 = target[train_i]
               
        model.fit(train1, y1)
        y1_pred = model.predict_proba(test1)
        folds_prediction.iloc[~train_i] = y1_pred[:,1]
    
        if metric_function:
            y1_test = target[~train_i]
            y_pred_class = model.predict(test1)
            metrics.append(metric_function(y1_test, y_pred_class))
    
    return folds_prediction, metrics

In [15]:
#Each base model will be fitted with the full training dataset and used to make predictions on the test set. These predictions will be stored in a new dataset

def make_predictions(model, X_train, X_test, target): 
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)
    y_pred = y_pred[:,1]
    return y_pred

In [16]:
beta = 2 #favors recall 

In [19]:
#%%time

metrics = pd.DataFrame(columns=["model", "accuracy", "F1" ,"precision", "recall", "Fbeta", "C_matrix"])

for classifier in classifiers: 
    
    model_name = classifier_name(classifier)
    print("--- Model:", model_name)
    
    l2_features = pd.DataFrame()
    l1_scores = []
    l1_test_predictions = pd.DataFrame()
    
    for feat_set in feat_to_combine: 
        print("- Processing Features:", feat_set)
        features_train, features_test = feature_sets[feat_set]

        #Separating ids and target from the datasets
        id_train = features_train["filename"]
        y_train = features_train["target"]
        folds_train = features_train["folds"]
        features_train = features_train.drop(["filename", "target", "folds"], axis=1)
        id_test = features_test["filename"]
        y_test = features_test["target"]
        features_test = features_test.drop(["filename", "target"], axis=1)

        #Feature transformation pipeline: only MinMaxScaling but more can be added later
        processing_pipeline = make_pipeline(MinMaxScaler())
        X_features = processing_pipeline.fit_transform(features_train)
        X_features_test = processing_pipeline.transform(features_test)
    
        #Making predictions for all the feature set
        folds_prediction, m1 = predict_by_folds(classifier, X_features, folds_train, y_train, f1_score)
        l2_features = l2_features.assign(new_prediction = folds_prediction).rename(columns = {"new_prediction": feat_set})
        l1_scores.append(np.mean(m1))
    
        #Training with the full train set and making predictions on the test set
        prediction = make_predictions(classifier, X_features, X_features_test, y_train)
        l1_test_predictions = l1_test_predictions.assign(new_prediction = prediction).rename(columns = {"new_prediction": feat_set})
        
    
    #Training the 2nd level classifier with the probability predictions of the level 1 classifiers as features
    classifier.fit(l2_features, y_train)
    y_pred = classifier.predict(l1_test_predictions)
    
    #Saving metrics
    m = {"model": model_name, 
         "accuracy": accuracy_score(y_test, y_pred), 
         "F1": f1_score(y_test, y_pred), 
         "precision": precision_score(y_test, y_pred), 
         "recall": recall_score(y_test, y_pred), 
         "Fbeta": fbeta_score(y_test, y_pred, beta), 
         "C_matrix": confusion_matrix(y_test, y_pred).ravel()} #tn, fp, fn, tp
    metrics = metrics.append(m, ignore_index=True)
    
    
    ###Average Ensemble predictions
    
    #Simple Average
    y_pred_av = l1_test_predictions.mean(axis=1)
    y_pred_av_class = y_pred_av > 0.5
    #Weighted average
    y_pred_w_av = (l1_test_predictions * l1_scores).sum(axis=1) / sum(l1_scores)
    y_pred_w_av_class = y_pred_w_av > 0.5
    
    m_av = {"model": model_name+"_Average", 
         "accuracy": accuracy_score(y_test, y_pred_av_class), 
         "F1": f1_score(y_test, y_pred_av_class), 
         "precision": precision_score(y_test, y_pred_av_class), 
         "recall": recall_score(y_test, y_pred_av_class),
         "Fbeta": fbeta_score(y_test, y_pred_av_class, beta), 
         "C_matrix": confusion_matrix(y_test, y_pred_av_class).ravel()}
    metrics = metrics.append(m_av, ignore_index=True)
    
    m_wav = {"model": model_name+"_Weighted_Average", 
         "accuracy": accuracy_score(y_test, y_pred_w_av_class), 
         "F1": f1_score(y_test, y_pred_w_av_class), 
         "precision": precision_score(y_test, y_pred_w_av_class), 
         "recall": recall_score(y_test, y_pred_w_av_class),
         "Fbeta": fbeta_score(y_test, y_pred_w_av_class, beta), 
         "C_matrix": confusion_matrix(y_test, y_pred_w_av_class).ravel()}
    metrics = metrics.append(m_wav, ignore_index=True)

--- Model: LogisticRegression
- Processing Features: FCTH
- Processing Features: JCD
- Processing Features: Tamura
--- Model: RandomForestClassifier
- Processing Features: FCTH
- Processing Features: JCD
- Processing Features: Tamura
--- Model: GaussianNB
- Processing Features: FCTH
- Processing Features: JCD
- Processing Features: Tamura


In [30]:
C_matrix = metrics["C_matrix"]
metrics = metrics.drop(["C_matrix"], axis=1)
metrics

model  accuracy        F1  precision  \
0                       LogisticRegression  0.883919  0.732774   0.842216   
1               LogisticRegression_Average  0.874872  0.703199   0.841408   
2      LogisticRegression_Weighted_Average  0.876124  0.706643   0.843629   
3                   RandomForestClassifier  0.869808  0.658711   0.923463   
4           RandomForestClassifier_Average  0.843633  0.566971   0.884899   
5  RandomForestClassifier_Weighted_Average  0.845795  0.577882   0.880399   
6                               GaussianNB  0.856037  0.630441   0.851954   
7                       GaussianNB_Average  0.809719  0.691342   0.574298   
8              GaussianNB_Weighted_Average  0.809833  0.691470   0.574475   

     recall     Fbeta  
0  0.648505  0.679774  
1  0.603988  0.640112  
2  0.607930  0.643910  
3  0.511941  0.562032  
4  0.417111  0.466425  
5  0.430095  0.479105  
6  0.500348  0.545363  
7  0.868305  0.787658  
8  0.868305  0.787725

In [31]:
#C_matrix: tn, fp, fn, tp
C_matrix = C_matrix.apply(pd.Series)
C_matrix.columns = ["TN", "FP", "FN", "TP"]
C_matrix["model"] = metrics["model"]
C_matrix = C_matrix.set_index(keys="model")
C_matrix

TN    FP    FN    TP
model                                                           
LogisticRegression                       12737   524  1516  2797
LogisticRegression_Average               12770   491  1708  2605
LogisticRegression_Weighted_Average      12775   486  1691  2622
RandomForestClassifier                   13078   183  2105  2208
RandomForestClassifier_Average           13027   234  2514  1799
RandomForestClassifier_Weighted_Average  13009   252  2458  1855
GaussianNB                               12886   375  2155  2158
GaussianNB_Average                       10485  2776   568  3745
GaussianNB_Weighted_Average              10487  2774   568  3745

In [32]:
metrics.sort_values(by=["accuracy", "F1"], ascending=False).to_csv(dpath+"metrics_late_fusion_v2.csv", index=False)

In [33]:
C_matrix.to_csv(dpath+"confusion_matrix_late_fusion_v2.csv")